<a href="https://colab.research.google.com/github/Mono-Blaine/CMPS-261-Project/blob/master/261_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import auc, accuracy_score, mean_squared_error, confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# load the dataset
# header=None prevents the first row from becoming column names
data = pd.read_csv('drive/MyDrive/HIGGS_train.csv', header=None)

<ipython-input-2-27c2f706f39b>:3: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('drive/MyDrive/HIGGS_train.csv', header=None)


# **Preprocessing**

**Dirty Data:**
* 490960V: error
* 490961V: s
* 261027I: "1.01"

In [3]:
# removes rows that aren't of type float. however, removes a lot so something wrong may be happening
# for i in range(29):
#     data = data[[not isinstance(value, str) for value in data[i]]]
#     data = data[[isinstance(value, float) for value in data[i]]]

# removes columns that aren't all float
# data2=data2.select_dtypes(include=['float64'])
# data2=data2.select_dtypes(exclude=['object'])

print(data.shape)

def filter(value):
    if isinstance(value, str) and not value.isnumeric():
        try:
            return float(value.replace('"','').replace('\'',''))
        except:
            return 0;
    return value

data = data.applymap(filter)

print(data.shape)

# get features
X = data.iloc[:, 1:]
# get output
y = data.iloc[:, 0]

(600000, 29)
(600000, 29)


# **Logistic Regression**

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# scales features
# result = (value - mean) / standardDeviation
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# fills missing values, if any
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

# create the model
model = LogisticRegression(max_iter=10000,  penalty='l2', solver='lbfgs')

# training
model.fit(X_train, y_train)

# predict the result of testing data
y_pred = model.predict(X_test)

# check model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# print(cross_val_score(model, X, y, cv=5, error_score='raise'))
# Accuracy: 0.6420222222222223

(420000, 28)
Accuracy: 0.6420166666666667


# **XGBoost**

In [ ]:
# create the XGBoost model
model = XGBClassifier(objective="binary:logistic",
                      random_state=42,
                      max_depth=6, # 0.7380666666666666
                      learning_rate=0.25)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# train the model
model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

# predict the result of testing data
y_pred = model.predict(X_test)

# check model performance
print("Accuracy Score:", accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred) 
print("Confusion Matrix")
print(cm)

# do K-fold (5-fold) cross-validation
kfold = KFold(n_splits=5, shuffle=True)
kf_cv_scores = cross_val_score(model, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

[0]	validation_0-logloss:0.65261
[1]	validation_0-logloss:0.62713
[2]	validation_0-logloss:0.60944
[3]	validation_0-logloss:0.59720
[4]	validation_0-logloss:0.58707
[5]	validation_0-logloss:0.57900
[6]	validation_0-logloss:0.57244
[7]	validation_0-logloss:0.56699
[8]	validation_0-logloss:0.56237
[9]	validation_0-logloss:0.55932
[10]	validation_0-logloss:0.55632
[11]	validation_0-logloss:0.55335
[12]	validation_0-logloss:0.55115
[13]	validation_0-logloss:0.54916
[14]	validation_0-logloss:0.54732
[15]	validation_0-logloss:0.54549
[16]	validation_0-logloss:0.54428
[17]	validation_0-logloss:0.54273
[18]	validation_0-logloss:0.54153
[19]	validation_0-logloss:0.54001
[20]	validation_0-logloss:0.53905
[21]	validation_0-logloss:0.53810
[22]	validation_0-logloss:0.53714
[23]	validation_0-logloss:0.53670
[24]	validation_0-logloss:0.53621
[25]	validation_0-logloss:0.53480
[26]	validation_0-logloss:0.53429
[27]	validation_0-logloss:0.53383
[28]	validation_0-logloss:0.53341
[29]	validation_0-loglos

# **Neural Networks**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = Sequential([Dense(240, activation="relu"),
                    Dense(120, activation="relu"),
                    Dense(50, activation="relu"),
                    Dense(30, activation="relu"),
                    Dense(15, activation="relu"),
                    Dense(1, activation="sigmoid")
                  ])

# checkpoint = ModelCheckpoint(monitor="val_loss", save_best_only=True, verbose=1)

optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=25)

model.evaluate(X_test,  y_test, verbose=2)

Epoch 1/25
13125/13125 [==============================] - 36s 2ms/step - loss: 0.6044 - accuracy: 0.6662
Epoch 2/25
13125/13125 [==============================] - 31s 2ms/step - loss: 0.5705 - accuracy: 0.6994
Epoch 3/25
13125/13125 [==============================] - 31s 2ms/step - loss: 0.5553 - accuracy: 0.7113
Epoch 4/25
13125/13125 [==============================] - 34s 3ms/step - loss: 0.5443 - accuracy: 0.7202
Epoch 5/25
13125/13125 [==============================] - 32s 2ms/step - loss: 0.5350 - accuracy: 0.7276
Epoch 6/25
13125/13125 [==============================] - 32s 2ms/step - loss: 0.5281 - accuracy: 0.7325
Epoch 7/25
13125/13125 [==============================] - 32s 2ms/step - loss: 0.5223 - accuracy: 0.7362
Epoch 8/25
13125/13125 [==============================] - 32s 2ms/step - loss: 0.5168 - accuracy: 0.7398
Epoch 9/25
13125/13125 [==============================] - 33s 3ms/step - loss: 0.5123 - accuracy: 0.7434
Epoch 10/25
13125/13125 [==============================

[nan, 0.7438833117485046]